In [1]:
from pyomo.environ import ConcreteModel, SolverFactory
                           
from idaes.core import FlowsheetBlock
                           
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
                           
from R32_bmimPF6_PR import configuration

from idaes.generic_models.unit_models import Flash

from idaes.core.util.model_statistics import degrees_of_freedom

import idaes.logger as idaeslog

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": False,
                           "has_pressure_change": False})

2021-08-16 12:01:38 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [3]:
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


In [4]:
m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(298)
m.fs.F101.inlet.pressure.fix(100000)

m.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.4)
m.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.6)
# m.fs.F101.vap_outlet.pressure.fix(399800)

# m.fs.F101.vap_outlet.temperature.fix(286)

In [5]:
degrees_of_freedom(m.fs.F101)

0

In [6]:
# Initialize the flash unit
m.fs.F101.initialize()

2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_in: Starting initialization
2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_in: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_in: Equilibrium temperature initialization completed.
2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_in: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2021-08-16 12:01:39 [INFO] idaes.init.fs.F101.control_volume.properties_out: Starting initialization
2021-08-16 12:01:40 [INFO] idaes.init.fs.F101.control_volume.properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-08-16 12:01:40 [INFO] idaes.init.fs.F101.control_volume.properties_out: Equilibrium temperatur

In [7]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [8]:
status = solver.solve(m, tee = True)

Ipopt 3.12.13: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. 

In [9]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance


------------------------------------------------------------------------------------
    Stream Table
                              Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                   1.0000      0.34834       0.65166  
    mole_frac_comp bmimPF6    0.60000   1.0000e-08       0.92073  
    mole_frac_comp R32        0.40000       1.0000      0.079270  
    temperature                298.00       298.00        298.00  
    pressure               1.0000e+05   1.0000e+05    1.0000e+05  


In [10]:
m.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block fs.properties
        
          Variables:
            PR_kappa : PR binary interaction parameters
                Size=4, Index=fs.properties.PR_kappa_index
                Key                    : Lower : Value    : Upper : Fixed : Stale : Domain
                        ('R32', 'R32') :  None :      0.0 :  None :  True :  True :  Reals
                    ('R32', 'bmimPF6') :  None : -0.03129 :  None :  True :  True :  Reals
                    ('bmimPF6', 'R32') :  None : -0.06828 :  None :  True :  True :  Reals
                ('bmimPF6', 'bmimPF6') :  None :      0.0 :  None :  True :  True :  Reals
        
          Objectives:
            None
        
          Constraints:
            None
        
          Blocks:
           

                    Key : Lower : Body                    : Upper
                    0.0 :   0.0 : -1.8614809960126877e-07 :   0.0
                pressure_balance : Size=1
                    Key : Lower : Body : Upper
                    0.0 :   0.0 :  0.0 :   0.0
            
              Blocks:
                Block fs.F101.control_volume.properties_in[0.0]
                
                  Variables:
                    flow_mol :  Total molar flowrate
                        Size=1, Index=None, Units=mol/s
                        Key  : Lower : Value : Upper : Fixed : Stale : Domain
                        None :     0 :     1 :  1000 :  True :  True : NonNegativeReals
                    mole_frac_comp : Mixture mole fractions
                        Size=2, Index=fs.properties.component_list
                        Key     : Lower : Value : Upper : Fixed : Stale : Domain
                            R32 :     0 :   0.4 :  None :  True :  True :  Reals
                       

                            ('Vap', 'Liq', 'R32') :   0.0 : -6.788373898558575e-07 :   0.0
                        ('Vap', 'Liq', 'bmimPF6') :   0.0 :  9.900000000000295e-07 :   0.0
                    eq_mole_frac_tbub : Size=1
                        Key            : Lower : Body : Upper
                        ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                    _teq_constraint_Vap_Liq : Size=1
                        Key  : Lower : Body : Upper
                        None :   0.0 :  0.0 :   0.0
                    equilibrium_constraint : Size=1
                        Key                   : Lower : Body                   : Upper
                        ('Vap', 'Liq', 'R32') :   0.0 : 2.5668255715372013e-11 :   0.0
            Block fs.F101.split
            
              Variables:
                _flow_mol_inlet_ref : Size=1, Index=fs._time
                    Key : Lower : Value : Upper : Fixed : Stale : Domain
                    0.0 :     0 :   1.0 :  1000 : False : Fa

In [11]:
# m.pprint()